# Analysis of Predicate Connectivity in Extracted Models

This section evaluates the connectivity between extracted models by analysing the appearance of predicates in preconditions and postconditions.

In [ ]:
import json

INPUT_FILE_PATH = '../examples/extracted_filtered_STRIPS.json'

with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as f:
    am_data = json.load(f)

EXPORT = False

To quantify predicate usage, a dictionary was constructed by counting the occurrences of each predicate within the model's preconditions and postconditions.

In [ ]:
def pred_key_to_str(pred_tuple):
    bool_value, predicate_name, args_themrole = pred_tuple
    return f"{bool_value}{predicate_name}({', '.join(args_themrole)})"

predicate_dict = {}
predicates = set()
for entry in am_data:
    class_id = entry.get('class_id', 'unknown')
    frames = entry.get('frames', [])
    for i, frame in enumerate(frames):
        preconditions = frame.get('preconditions', [])
        postconditions = frame.get('postconditions', [])
        class_id_with_frame = f"{class_id}-frame-{i}"
        
        # Check all predicates in preconditions
        for condition in preconditions:
            name = condition[1]
            if name not in predicate_dict:
                predicate_dict[name] = {
                    'pre_class_id': [class_id_with_frame],
                    'post_class_id': []
                }
            else:
                predicate_dict[name]['pre_class_id'].append(
                    class_id_with_frame)

        # Check all predicates in postconditions
        for condition in postconditions:
            name = condition[1]
            if name not in predicate_dict:
                predicate_dict[name] = {
                    'pre_class_id': [],
                    'post_class_id': [class_id_with_frame],
                }
            else:
                predicate_dict[name]['post_class_id'].append(
                    class_id_with_frame)
         
for key in predicate_dict:
    predicate_dict[key]['preconditions_count'] = \
        len(predicate_dict[key]['pre_class_id'])
    predicate_dict[key]['postconditions_count'] = \
        len(predicate_dict[key]['post_class_id'])

if EXPORT:
    with open("predicate_dict.json", 'w', encoding='utf-8') as f:
        json.dump(predicate_dict, f, indent=2)

Based on this collected data, the predicates were categorised and counted according to whether they appeared in both preconditions and postconditions, only in preconditions, or exclusively in postconditions.

The analysis indicates that a total of 80 predicates are utilised in the filtered action models. Among these, 48 appeared in both preconditions and postconditions. Additionally, 27 predicates were found only in postconditions, and 5 were present only in preconditions.

In [13]:
def count_empty_condtion(data: dict, condition_type: str) -> tuple:
    """
    Filter out entries with the specified condition type count as 0.
    """
    empty_condition_list = [
        (key, value.get('preconditions_count', -1),
         value.get('postconditions_count', -1))
        for key, value in data.items()
        if value.get(condition_type, -1) == 0
    ]
    return empty_condition_list


def count_non_empty_condition(data: dict) -> tuple:
    """
    Filter out entries with the specified condition type count as non-zero.
    """
    non_empty_condition_list = [
        (key, value.get('preconditions_count', -1),
         value.get('postconditions_count', -1))
        for key, value in data.items()
        if value.get('preconditions_count', -1) > 0 and value.get('postconditions_count', -1) > 0
    ]
    return non_empty_condition_list


no_precondition_list = count_empty_condtion(predicate_dict, 'preconditions_count')
no_postcondition_list = count_empty_condtion(predicate_dict, 'postconditions_count')
non_empty_condition_list = count_non_empty_condition(predicate_dict)

print(f"Number of predicates used in extracted model: {len(predicate_dict)}")
print(f"Number of predicates appear in both preconditions and postconditions: {len(non_empty_condition_list)}")
print(f"Number of predicates not appear in preconditions: {len(no_precondition_list)}")
print(f"Number of predicates not appear in postconditions: {len(no_postcondition_list)}")

Number of predicates used in extracted model: 80
Number of predicates appear in both preconditions and postconditions: 48
Number of predicates not appear in preconditions: 27
Number of predicates not appear in postconditions: 5
